# 07장 정규표현식  
---

## 07-1  정규 표현식 살펴보기
---

정규 표현식(Regular Expressions)은 복잡한 문자열을 처리할 때 사용하는 기법으로, 파이썬만의 고유 문법이 아니라 문자열을 처리하는 모든 곳에서 사용한다. 정규 표현식을 배우는 것은 파이썬을 배우는 것과는 또 다른 영역의 과제이다.

### 1.정규 표현식은 왜 필요한가?  
---

다음과 같은 문제가 주어졌다고 가정해 보자.  

`주민등록번호를 포함하고 있는 텍스트가 있다. 이 텍스트에 포함된 모든 주민등록번호의 뒷자리를 * 문자로 변경해 보자.`  

우선 정규식을 전혀 모르면 다음과 같은 순서로 프로그램을 작성해야 할 것이다.

1. 전체 텍스트를 공백 문자로 나눈다(split).  

2. 나뉜 단어가 주민등록번호 형식인지 조사한다.  

3. 단어가 주민등록번호 형식이라면 뒷자리를 *로 변환한다.  

4. 나뉜 단어를 다시 조립한다.  

이를 구현한 코드는 아마도 다음과 같을 것이다.

In [1]:
data = """
park 800905-1049118
kim  700905-1059119
"""

result = []
for line in data.split("\n"):
    word_result = []
    for word in line.split(" "):
        if len(word) == 14 and word[:6].isdigit() and word[7:].isdigit():
            word = word[:6] + "-" + "*******"
        word_result.append(word)
    result.append(" ".join(word_result))
print("\n".join(result))


park 800905-*******
kim  700905-*******



In [2]:
import re 

data = """
park 800905-1049118
kim  700905-1059119
"""

pat = re.compile("(\d{6})[-]\d{7}")
print(pat.sub("\g<1>-*******", data))


park 800905-*******
kim  700905-*******



## 07-2 정규 표현식 시작하기  
---

### 1. 메타문자  
---

- 메타문자    
`>^$*+?{}[]\|()`

- 문자 클래스  
[ ] 사이의 문자들과 매치  

[abc]  
`a`는 정규식과 일치하는 문자인 `a`가 있으므로 매치  
`before`는 정규식과 일치하는 문자인 `b`가 있으므로 매치  
`dude`는 정규식과 일치하는 문자가  `a,b,c` 중 어느 하나도 포함하고 있지 않으므로 매치되지 않음

-:form to  
[a-c]=[abc],[a-zA-Z]  
^:반대(not)  
[^0-9]

- Dot(.)  
`\n`을 제외한 모든 문자와 매치  
`a.b` $\rightarrow$ `a + 모든 문자 + b` 
`a[.]b`$\rightarrow$ `a + . + b`  <span style="color:blue">[]안에 있는 .은 그 자체를 의미</span>    

- * : 반복  
`ca*t` a가 $0$ ~ $\infty$ 반복 가능!    
사실...2억 번 정도로 제한  
내가 2억 번까지 할까 싶긴 한데 알아둬야지!

- + : 반복  
`ca+t` a가 1번 이상 반복 가능!

- {m,n},? : 반복    
만약 반복을 1회,3회 이렇게 제한을 두고 싶르면 {}를 사용하여 횟수 고정 가능  
{ㅡm,n} m부터 n까지 매치  
?:{0,1}을 의미

### 2. 정규표현식을 지원하는  re모듈    
---

In [2]:
import re
p=re.compile('ab*')

### 3.정규식을 이용한 문자열 검색  
---

컴파일된 패턴 객체를 사용하여 문자열 검색을 수행  
4가지 method 제공  

|Method|목적|  
|---|------|
|match()|문자열의 처음부터 정규식과 매치되는지 조사|
|search()|문자열 전체를 검색하여 정규식과 매치되는지 조사|
|findall()|정규식과 매치되는 모든 문자열(substring)을 리스트로|
|finditer()|정규식과 매치되는 모든 문자열(substring)을 반복 가능한 객체로|  

match, search는 정규식과 매치될 때는 match 객체(정규식의 검색 결과로 돌려주는 객체)를 돌려주고, 매치되지 않을 때는 None을

- 패턴 만들기  
소문자 1번 이상 반복

In [3]:
import re
p=re.compile('[a-z]+')

- match

In [4]:
m=p.match("python")
print(m)

<re.Match object; span=(0, 6), match='python'>


In [5]:
m=p.match("3 python")
print(m)

None


- search

In [6]:
m=p.search("python")
print(m)

<re.Match object; span=(0, 6), match='python'>


In [7]:
m=p.search("3 python")
print(m)

<re.Match object; span=(2, 8), match='python'>


search는 문자열부터 그래서 3 이후의 p부터

- findall

In [11]:
result=p.findall("Life is too short")
print(result)

['ife', 'is', 'too', 'short']


- finditer

In [12]:
result=p.finditer("Life is too short")
print(result)

In [13]:
for r in result:print(r)

<re.Match object; span=(1, 4), match='ife'>
<re.Match object; span=(5, 7), match='is'>
<re.Match object; span=(8, 11), match='too'>
<re.Match object; span=(12, 17), match='short'>


finditer는 findall과 동일하지만 그 결과로 반복 가능한 객체(iterator object)를 돌려준다. 반복 가능한 객체가 포함하는 각각의 요소는 match 객체

### 4.match 객체의 메서드  
---

|method|목적|
|---|---|
|group()|매치된 문자열|
|start()|매치된 문자열의 시작 위치|
|end()|매치된 문자열의 끝 위치|
|span()|매치된 문자열의 (시작, 끝)에 해당하는 튜플|

In [8]:
m = p.match("python")
m.group()

'python'

In [9]:
m.start()

0

In [10]:
m.end()

6

In [11]:
m.span()

(0, 6)

In [12]:
m = p.search("3 python")
m.group()

'python'

In [13]:
m.start()

2

In [14]:
m.end()

8

In [15]:
m.span()

(2, 8)

지금까지 우리는 `re.compile`을 사용하여 컴파일된 패턴 객체로 그 이후의 작업을 수행  
좀 축약한 형태로 사용할 수 있는 방법은 다음과 같음

In [18]:
m = re.match('[a-z]+', "python")
m

<re.Match object; span=(0, 6), match='python'>

### 5.컴파일 옵션  
---

- DOTALL(S)  . 이 줄바꿈 문자를 포함하여 모든 문자와 매치
- IGNORECASE(I)  대소문자에 관계없이 매치  
- MULTILINE(M)  여러줄과 매치 (^, $ 메타문자의 사용과 관계가 있는 옵션이다)  
- VERBOSE(X)  verbose 모드를 사용 (정규식을 보기 편하게 만들수 있고 주석등을 사용할 수 있게된다.)

 `re.DOTALL`처럼 전체 옵션 이름을 써도 되고 `re.S`처럼 약어도 가능

`.` 메타 문자는 줄바꿈 문자(`\n`)를 제외한 모든 문자와 매치되는 규칙이 있다. 만약`\n` 문자도 포함하여 매치하고 싶다면 `re.DOTALL`(`re.S`) 옵션을 사용해 정규식을 컴파일

In [19]:
import re
p = re.compile('a.b')
m = p.match('a\nb')
print(m)

None


In [20]:
p = re.compile('a.b', re.DOTALL)
m = p.match('a\nb')
print(m)

<re.Match object; span=(0, 3), match='a\nb'>


In [24]:
m = re.match('a.b', "a\nb",re.DOTALL)
print(m)

<re.Match object; span=(0, 3), match='a\nb'>


In [25]:
m = re.match('[a-z]+','python' ,re.I)
print(m)

<re.Match object; span=(0, 6), match='python'>


In [26]:
m = re.match('[a-z]+','Python' ,re.I)
print(m)

<re.Match object; span=(0, 6), match='Python'>


 `^`는 문자열의 처음을 의미하고, `$`는 문자열의 마지막을 의미  
`^python`인 경우 문자열의 처음은 항상 python으로 시작해야 매치되고, 만약 정규식이 `python$`이라면 문자열의 마지막은 항상 python으로 끝나야 매치된다는 의미

In [27]:
import re
p = re.compile("^python\s\w+")

data = """python one
life is too short
python two
you need python
python three"""

print(p.findall(data))

['python one']


 `^python\s\w+`은 python이라는 문자열로 시작하고 그 뒤에 whitespace, 그 뒤에 단어가 와야 한다는 의미  
`^` :문자열의 첫번째만

In [28]:
import re
p = re.compile("^python\s\w+", re.MULTILINE)

data = """python one
life is too short
python two
you need python
python three"""

print(p.findall(data))

['python one', 'python two', 'python three']


문자열 전체의 처음이 아니라 각 라인의 처음으로 인식시키고 싶은 경우

In [29]:
charref = re.compile(r'&[#](0[0-7]+|[0-9]+|x[0-9a-fA-F]+);')

In [30]:
charref = re.compile(r"""
 &[#]                # Start of a numeric entity reference
 (
     0[0-7]+         # Octal form
   | [0-9]+          # Decimal form
   | x[0-9a-fA-F]+   # Hexadecimal form
 )
 ;                   # Trailing semicolon
""", re.VERBOSE)

문자열에 사용된 whitespace는 컴파일할 때 제거된다(단 [ ] 안에 사용한 whitespace는 제외

### 6.백슬래시 문제  
---

정규식 엔진에 `\\`문자를 전달하려면 파이썬은 `\\\\`처럼 백슬래시를 4개나 사용

정규식 문자열 앞에 r 문자를 삽입하면 이 정규식은 Raw String 규칙에 의하여 백슬래시 2개 대신 1개만 써도 2개를 쓴 것과 동일한 의미를 갖게 된다.

## 07-3강력한 정규 표현식의 세계로  
---

### 1.메타문자  
---

- |  
---

 `+, *, [], {}` 등의 메타문자는 매치가 진행될 때 현재 매치되고 있는 문자열의 위치가 변경된다(보통 소비된다고 표현한다). 하지만 이와 달리 문자열을 소비시키지 않는 메타 문자도 있다. 이번에는 이런 문자열 소비가 없는(zerowidth assertions) 메타 문자에 대해 살펴보자.

In [41]:
p = re.compile('Crow|Servo') #or 
m = p.match('CrowHello')
print(m)

<re.Match object; span=(0, 4), match='Crow'>


- ^  
---

`^` 메타 문자는 문자열의 맨 처음과 일치함을 의미한다. 앞에서 살펴본 컴파일 옵션 `re.MULTILINE`을 사용할 경우에는 여러 줄의 문자열일 때 각 줄의 처음과 일치하게 된다.

In [42]:
print(re.search('^Life', 'Life is too short'))

<re.Match object; span=(0, 4), match='Life'>


In [43]:
 print(re.search('^Life', 'My Life'))

None


<span style="color:blue">처음 위치가 아닌 경우에는 매치되지 않음</span>

- $  
---

In [44]:
print(re.search('short$', 'Life is too short'))

<re.Match object; span=(12, 17), match='short'>


In [45]:
print(re.search('short$', 'Life is too short, you need python'))

None


- \A  
---

`\A`는 문자열의 처음과 매치됨을 의미  
`re.MULTILINE` 옵션을 사용할 경우 `^`은 각 줄의 문자열의 처음과 매치되지만 `\A`는 줄과 상관없이 전체 문자열의 처음하고만 매치

- \Z  
---

`\Z`는 문자열의 끝과 매치됨을 의미  
역시 `\A`와 동일하게 `re.MULTILINE` 옵션을 사용할 경우 `$` 메타 문자와는 달리 전체 문자열의 끝과 매치

- \b  
---

- \B  
---

### 2.그루핑  
---

### 3. 전방 탐색  
---

### 4.문자열 바꾸기  
---

sub 메서드를 사용하면 정규식과 매치되는 부분을 다른 문자로 쉽게 바꿀 수 있다.

In [36]:
p = re.compile('(blue|white|red)')
p.sub('colour', 'blue socks and red shoes')

'colour socks and colour shoes'

sub 메서드의 첫 번째 매개변수는 "바꿀 문자열(replacement)"이 되고, 두 번째 매개변수는 "대상 문자열"이 된다. 위 예에서 볼 수 있듯이 blue 또는 white 또는 red라는 문자열이 colour라는 문자열로 바뀌는 것을 확인할 수 있다.  
딱 한 번만 하고 싶으면 count=1 이케하면 됨

- [sub 메서드와 유사한 subn 메서드]  
반환 결과를 튜플로 돌려준다는 차이  
돌려준 튜플의 첫 번째 요소는 변경된 문자열이고, 두 번째 요소는 바꾸기가 발생한 횟수 

In [37]:
p = re.compile('(blue|white|red)')  
p.subn( 'colour', 'blue socks and red shoes')

('colour socks and colour shoes', 2)

In [38]:
p = re.compile(r"(?P<name>\w+)\s+(?P<phone>(\d+)[-]\d+[-]\d+)")
print(p.sub("\g<phone> \g<name>", "park 010-1234-1234"))

010-1234-1234 park


In [39]:
print(p.sub("\g<2> \g<1>", "park 010-1234-1234")) #참조 번호

010-1234-1234 park


In [40]:
def hexrepl(match):
    value = int(match.group())
    return hex(value)
p = re.compile(r'\d+')
p.sub(hexrepl, 'Call 65490 for printing, 49152 for user code.')

'Call 0xffd2 for printing, 0xc000 for user code.'

hexrepl 함수는 match 객체(위에서 숫자에 매치되는)를 입력으로 받아 16진수로 변환하여 돌려주는 함수  
sub의 첫 번째 매개변수로 함수를 사용할 경우 해당 함수의 첫 번째 매개변수에는 정규식과 매치된 match 객체가 입력되고 매치되는 문자열은 함수의 반환 값으로 바뀌게 된다.

### 5.Greedy vs Non-Greedy  
---

In [32]:
s = '<html><head><title>Title</title>'  
len(s)

32

In [33]:
print(re.match('<.*>', s).span())

(0, 32)


In [34]:
print(re.match('<.*>', s).group())

<html><head><title>Title</title>


`<.*>` 정규식의 매치 결과로 `<html>` 문자열을 돌려주기를 기대했을 것이다.   
하지만 `*` 메타 문자는 매우 탐욕스러워서 매치할 수 있는 최대한의 문자열인 `<html><head><title>Title</title>` 문자열을 모두 소비해 버렸다.   
어떻게 하면 이 탐욕스러움을 제한하고 `<html>` 문자열까지만 소비하도록 막을 수 있을까?  
 `non-greedy` 문자인 `?`를 사용하면 `*`의 탐욕을 제한 가능  

In [35]:
print(re.match('<.*?>', s).group())

<html>


 `?`는 `*?`, `+?`, `??`, `{m,n}?`와 같이 사용할 수 있다.   
 가능한 한 가장 최소한의 반복을 수행하도록 도와주는 역할